# Apply Benchmark alg to Instances

In [1]:
# %% import packages
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid

sys.path.append("./source/")

from source.agent_Kano import HDoC_Kano, LUCB_G_Kano
from source.agent_lilHDoC import lilHDoC
from source.agent_AdaptedTaS_new import Adapted_TaS_new
from source.agent_MS import MS

from source.utils import Linear, Unique_Qualified, OneQuarter_Qualified, AllWorse, AllGood, HalfGood, Experiment
from source.env import Environment_Gaussian

Delta = 0.15
Delta_str = str(Delta).replace(".", "p")
filefolder = "./Numeric-Record-Delta_"+ Delta_str + "/"
print(filefolder)
if not os.path.isdir(filefolder):
    os.mkdir(filefolder)

./Numeric-Record-Delta_0p15/


In [ ]:
# %% specify parameter types
instance_type_ = [Linear, Unique_Qualified, OneQuarter_Qualified, AllWorse, AllGood, HalfGood]
agent_class_ = [HDoC_Kano, LUCB_G_Kano, lilHDoC]

K_ = [10, 150, 100, 50, 40, 30, 20, 200]
delta_ = [0.001, 0.0001]

env_ = [Environment_Gaussian]
n_exp = 1000
xi = 0.5

experiment_grid = ParameterGrid(
    {
        "instance_type": instance_type_,
        "agent": agent_class_,
        "env": env_,
        "delta": delta_,
        "K": K_,
    }
)

In [ ]:
for exp_setting_id, experiment in enumerate(experiment_grid):
    K = experiment["K"]
    delta = experiment["delta"]
    if experiment["instance_type"]==AllWorse:
        inst_fun = experiment["instance_type"]
        rlist = inst_fun(K, Delta=0.25)
    else:
        inst_fun = experiment["instance_type"]
        rlist = inst_fun(K, Delta=Delta)
    

    env_class = experiment["env"]
    agent_class = experiment["agent"]

    numeric_record = pd.DataFrame(
        columns=[
            "K",
            "n_experiment",
            "delta",
            "instance_type",
            "env",
            "agent",
            "success_rate",
            "reward_array",
            "mean_stop_times",
            "std_stop_times",
            "forced-stop",
            "count_exceed_stop",
            "count_success",
        ]
    )
    stop_benchmark = 1e8
        
    if inst_fun.__name__ == "AllWorse" and agent_class.__name__=="MS":
        continue
        
    mean_stop_time, std_stop_time, mean_success, _, _, count_exceed_stop, count_success = Experiment(
        rlist=rlist,
        delta=delta,
        K=K,
        xi=xi,
        env_class=env_class,
        agent_class=agent_class,
        random_seed_for_start=42,
        n_exp=n_exp,
        stop_benchmark=stop_benchmark,
        disable_tqdm=False,
    )
    filename = filefolder + agent_class.__name__ + ".csv"
    numeric_record.loc[0] = np.array(
        [
            K,
            n_exp,
            delta,
            inst_fun.__name__,
            env_class.__name__,
            agent_class.__name__,
            mean_success,
            np.array2string(rlist, threshold=11e3),
            mean_stop_time,
            std_stop_time,
            stop_benchmark,
            count_exceed_stop,
            count_success
        ],
        dtype=object,
    )
    numeric_record.to_csv(filename, mode="a", index=False, header=not os.path.exists(filename))
    print(
        f"{inst_fun.__name__}, {agent_class.__name__}, {env_class.__name__}, {K},{delta}, {exp_setting_id+1}/{len(experiment_grid)} done, stop {mean_stop_time}, success {mean_success}"
    )